
# Transfer Learning for Computer Vision Tutorial

Adapted from: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html


**Author**: [Sasank Chilamkurthy](https://chsasank.github.io)

In this tutorial, you will learn how to train a convolutional neural network for image classification using transfer learning. You can read more about the transfer learning at [cs231n notes](https://cs231n.github.io/transfer-learning/)_ 
Quoting these notes,

    In practice, very few people train an entire Convolutional Network
    from scratch (with random initialization), because it is relatively
    rare to have a dataset of sufficient size. Instead, it is common to
    pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
    contains 1.2 million images with 1000 categories), and then use the
    ConvNet either as an initialization or a fixed feature extractor for
    the task of interest.

## Resnet18 for Transfer Learning

### What is Resnet18

A ```resnet18``` is a pretrained deep feed forward neural network with 17 convolution layers and 1 fully connected layer. According to [this thesis](ResnetRef.pdf), introducing feed forward shortcuts to a deeply stacked network can further increase the accuracy of the model. Such feed forward shortcuts can counter the problem with gradient vanishing by adding the input of the layer block to its output. A layer block is a pair of convolution layers in different sizes. During training, if the convolution layers in the layer block is not helpful, its weights can be lowered and thus the output of the layer block can be more dependent to its input, essentially bypassing the convolution layers within.

The original ```Resnet18``` is designed to classify 1000 different labels, therefor its final layer is a fully connected layer taking in the features from the previous convolution layer and outputs 1000 scores for each label. In this demo we only have to classify 2 different lables, bees or ants. The final fully connected layer should be modified to satisfy our needs.

![fff](img/Resnet18.svg)

In [3]:
# License: BSD
# Author: Sasank Chilamkurthy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from os.path import join

## Load Data

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   [here](https://download.pytorch.org/tutorial/hymenoptera_data.zip)
   and extract it to the current directory.



In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
DATA_DIR:str = join('SummerVacationMeeting', 'W4_TransferLearning', 'hymenoptera_data')
BATCH_SIZE = 4

data_transforms:dict[str, transforms.Compose] = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

""" 
create dataset objects for training & validation datasets and store them into image_datasets dictionary
torchvision.datasets.ImageFolder inherits torch.utils.data.Dataset class 
"""
training_dataset = ImageFolder(
    root = join(DATA_DIR, 'train'),
    transform = data_transforms['train']
)
val_dataset = ImageFolder(
    root = join(DATA_DIR, 'val'),
    transform = data_transforms['val']
)
image_datasets: dict[str, ImageFolder] = {'train': training_dataset, 'val': val_dataset}


""" create dataloader objects for training & validation dataloaders and store them into dataloaders dictionary """
train_dataloader = DataLoader(training_dataset, 
    batch_size = BATCH_SIZE, 
    shuffle = True, 
    num_workers = 4)
val_dataloader = DataLoader(val_dataset, 
    batch_size = BATCH_SIZE, 
    shuffle = True, 
    num_workers = 4)
dataloaders:dict[str, DataLoader] = {'train': train_dataloader, 'val': val_dataloader}

dataset_sizes = {train_val: len(image_datasets[train_val]) for train_val in ['train', 'val']}
class_names = image_datasets['train'].classes

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Train All Layers and Weights in Resnet18

Pytorch provides a ```models.resnet18``` class and its pretrained weights. The following cell makes ```model_ft``` an instance of ```resnet18``` and replaces its pretrained final fully connected layer with our own custom fully connected layer that takes in the same number of inputs from the previous convolution layer and outputs the scores for the 2 labels. This new fully conneced layer is to be trained.

![entire resnet to be trained](img/Resnet18_learn_all.svg)

In [5]:
model_ft = models.resnet18(weights = "IMAGENET1K_V1")
"""
The possible weight options for resnet18 are "DEFAULT", "IMAGENET1K_V1" or None
If the weight argument is left blank or set to None, no weights for resent18 will be loaded
According to https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py#L10 option "DEFAULT" and "IMAGENET_1K_V!" loades the same set of weights
"""

""" 
Replace the last fully connected layer in Resnet18 that takes in model_ft.fc.in_features number of features and out put 1000 class scores with our own custom
fully connected layer that takes in model_ft.fc.in_features features and outputs 2 class scores. 
"""
model_ft.fc = nn.Linear(model_ft.fc.in_features, 2)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\yusin/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


### Train and evaluate

It should take around 15-25 min on CPU. On GPU though, it takes less than a minute.

The training loop is written in a [seperate file](train_and_test_model.py). Both the training loop provided by the tutorial and the denestified version are present. For some reason, models trained using the denestified version performs significantly worse than the original versoin?

In [6]:
# FIXME: the train_model function in the tutorial performs better in terms of accuracy
# from train_and_test_model import my_train_model
# model_ft = my_train_model(model_ft, criterion, dataloaders, dataset_sizes, optimizer_ft, exp_lr_scheduler, DEVICE)

In [7]:
from train_and_test_model import tutorial_train_model
model_ft = model_ft.to(DEVICE)
model_ft = tutorial_train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, dataset_sizes, DEVICE)

Epoch 0/24
----------
train Loss: 0.7319 Acc: 0.6107
val Loss: 0.2766 Acc: 0.9085

Epoch 1/24
----------
train Loss: 0.3875 Acc: 0.8566
val Loss: 0.2595 Acc: 0.9020

Epoch 2/24
----------
train Loss: 0.6814 Acc: 0.7582
val Loss: 0.2168 Acc: 0.8954

Epoch 3/24
----------
train Loss: 0.4619 Acc: 0.7992
val Loss: 0.4713 Acc: 0.7974

Epoch 4/24
----------
train Loss: 0.5333 Acc: 0.8033
val Loss: 0.2376 Acc: 0.9150

Epoch 5/24
----------
train Loss: 0.5661 Acc: 0.8033
val Loss: 0.3659 Acc: 0.8758

Epoch 6/24
----------
train Loss: 0.5943 Acc: 0.7869
val Loss: 0.4601 Acc: 0.8693

Epoch 7/24
----------
train Loss: 0.5597 Acc: 0.8115
val Loss: 0.2945 Acc: 0.8758

Epoch 8/24
----------
train Loss: 0.3350 Acc: 0.8648
val Loss: 0.2994 Acc: 0.9020

Epoch 9/24
----------
train Loss: 0.3644 Acc: 0.8607
val Loss: 0.2553 Acc: 0.9150

Epoch 10/24
----------
train Loss: 0.3176 Acc: 0.8648
val Loss: 0.2704 Acc: 0.9020

Epoch 11/24
----------
train Loss: 0.2673 Acc: 0.8730
val Loss: 0.2956 Acc: 0.8889

Ep

train Loss: 0.2762 Acc: 0.9016


## Resnet as Fixed Feature Extractor

Retraining all the layers, including those that were pretrained in ```resnet18``` might be unnecessary. The first 17 layers in this network are convolution layers. These convolution layers act like feature extractors, or filters, operating on given images. Their parameters and weights might have been optimized when we load them and don't require futher training.

What requires training is the custom fully connected layer we have just added to this network. To train only the final layer and not train the others, we need to freeze all the network except the final one. We need to set ```requires_grad = False``` to freeze the parameters so that the gradients are not computed in ```backward()```. 
You can read more about this in the documentation
[here](https://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward)_.

![only train the last layer resnet](img/Resnet18_learn_only_last_layer.svg)


In [ ]:
model_conv = torchvision.models.resnet18(weights='IMAGENET1K_V1')
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:

# FIXME: train_model function in the tutorial performs better in terms of accuracy
# from train_and_test_model import my_train_model
# model_conv = my_train_model(model_conv, criterion, dataloaders, dataset_sizes, optimizer_conv, exp_lr_scheduler, DEVICE)

In [ ]:
from train_and_test_model import tutorial_train_model
model_conv = tutorial_train_model(model_conv, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, dataset_sizes, DEVICE)